<a href="https://colab.research.google.com/github/Kostia9/Data-Analysis-2025/blob/main/Lab3/Lab3_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os, json
from google.colab import userdata

os.makedirs(os.path.expanduser("~/.kaggle"), exist_ok=True)

os.environ["KAGGLE_JSON"] = userdata.get("KAGGLE_JSON")
kaggle_json = os.environ['KAGGLE_JSON']
kaggle_token = json.loads(kaggle_json)

# Записати у файл kaggle.json
kaggle_path = os.path.expanduser("~/.kaggle/kaggle.json")
with open(kaggle_path, "w") as f:
    json.dump(kaggle_token, f)
os.chmod(kaggle_path, 0o600)

# 4) Встановити kaggle і завантажити файли
!pip install -q kaggle

!kaggle datasets download -d blastchar/telco-customer-churn
!unzip telco-customer-churn.zip

Dataset URL: https://www.kaggle.com/datasets/blastchar/telco-customer-churn
License(s): copyright-authors
telco-customer-churn.zip: Skipping, found more recently modified local copy (use --force to force download)
Archive:  telco-customer-churn.zip
replace WA_Fn-UseC_-Telco-Customer-Churn.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n


In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import StratifiedKFold
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score, roc_auc_score, average_precision_score
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier

In [3]:
df = pd.read_csv("/content/WA_Fn-UseC_-Telco-Customer-Churn.csv")
print(df.shape)
df.head()

(7043, 21)


,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [4]:
df = df.drop('customerID', axis=1)
df['TotalCharges'] = pd.to_numeric(df['TotalCharges'], errors='coerce')
df = df.dropna()
X = df.drop('Churn', axis=1)
y = df["Churn"].map({"No":0, "Yes":1}).astype(int)

# train/test split (stratified)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=67, stratify=y
)

# train/val split (stratified)
X_tr_raw, X_val_raw, y_tr, y_val = train_test_split(
    X_train, y_train, test_size=0.2, random_state=67, stratify=y_train
)

cols = X_train.columns.tolist()
num_cols = ['tenure', 'MonthlyCharges', 'TotalCharges']
cat_cols = [c for c in X.columns if c not in num_cols]

num_scaler = StandardScaler().fit(X_tr_raw[num_cols])
ohe = OneHotEncoder(handle_unknown="ignore", drop="if_binary").fit(X_tr_raw[cat_cols])

def transform(df):
    Xn = num_scaler.transform(df[num_cols])
    Xc = ohe.transform(df[cat_cols]).toarray()
    return np.hstack([Xn, Xc])

Xtr  = transform(X_tr_raw)
Xval = transform(X_val_raw)
Xte  = transform(X_test)

print("X_train:", Xtr.shape, "X_val:", Xval.shape, "X_test:", Xte.shape)
print("y_train:", y_tr.shape, "y_val:", y_val.shape, "y_test:", y_test.shape)

X_train: (4500, 40) X_val: (1125, 40) X_test: (1407, 40)
y_train: (4500,) y_val: (1125,) y_test: (1407,)


# Build a Fully Connected Neural Network with PyTorch

In [5]:
Xtr_t  = torch.from_numpy(Xtr).float()
Xval_t = torch.from_numpy(Xval).float()
Xte_t  = torch.from_numpy(Xte).float()

ytr_t  = torch.from_numpy(y_tr.values.astype(np.float32)).unsqueeze(1)
yval_t = torch.from_numpy(y_val.values.astype(np.float32)).unsqueeze(1)
yte_t  = torch.from_numpy(y_test.values.astype(np.float32)).unsqueeze(1)

train_loader = DataLoader(TensorDataset(Xtr_t, ytr_t), batch_size=512, shuffle=True)
val_loader   = DataLoader(TensorDataset(Xval_t, yval_t), batch_size=4096, shuffle=False)

In [6]:
# Define the Fully Connected Neural Network
class Net(nn.Module):
    def __init__(self, input_size):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(input_size, 128)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 1) # Output layer for binary classification

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.relu(x)
        x = self.fc3(x)
        return x

In [7]:
def train(model, train_loader, criterion, optimizer):
    model.train()
    tr_loss = 0.0
    for xb, yb in train_loader:
        optimizer.zero_grad(set_to_none=True)
        logits = model(xb)
        loss = criterion(logits, yb)
        loss.backward()
        nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        tr_loss += loss.item() * xb.size(0)
    return tr_loss / len(train_loader.dataset)

In [8]:
def validate(model, val_loader, criterion):
    model.eval()
    val_loss = 0.0
    val_probs_all = []
    val_true_all  = []
    with torch.no_grad():
        for xb, yb in val_loader:
            logits = model(xb)
            loss = criterion(logits, yb)
            val_loss += loss.item() * xb.size(0)
            val_probs_all.append(torch.sigmoid(logits).detach().cpu().numpy().ravel())
            val_true_all.append(yb.detach().cpu().numpy().ravel())

    val_loss /= len(val_loader.dataset)
    val_probs = np.concatenate(val_probs_all)
    val_true  = np.concatenate(val_true_all)
    val_auc = roc_auc_score(val_true, val_probs)
    val_ap  = average_precision_score(val_true, val_probs)

    return val_loss, val_auc, val_ap


In [9]:
model = Net(Xtr.shape[1])
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

patience = 8
bad = 0
best_metric = -1.0
best_state = None
num_epochs = 500

for epoch in range(num_epochs):
    # --- train ---
    tr_loss = train(model, train_loader, criterion, optimizer)

    # --- validate ---
    val_loss, val_auc, val_ap = validate(model, val_loader, criterion)

    if (epoch+1) % 10 == 0:
        print(f"Epoch [{epoch+1}/{num_epochs}], Train Loss: {tr_loss:.4f}, Val Loss: {val_loss:.4f}, "
              f"Val ROC-AUC: {val_auc:.4f}, Val AP: {val_ap:.4f}")

    # --- early stopping по PR-AUC ---
    if val_ap > best_metric + 1e-5:
        best_metric = val_ap
        best_state = {k: v.detach().cpu().clone() for k, v in model.state_dict().items()}
        bad = 0
    else:
        bad += 1
        if bad >= patience:
            print(f"Early stopping at epoch {epoch+1}. Best Val AP: {best_metric:.4f}")
            break

print(f"\nTraining finished. Best Validation AP: {best_metric:.4f}")

# Load the best model state
if best_state:
    model.load_state_dict(best_state)
    print("Loaded best model state.")

Epoch [10/500], Train Loss: 0.5826, Val Loss: 0.5769, Val ROC-AUC: 0.7695, Val AP: 0.5311
Epoch [20/500], Train Loss: 0.4826, Val Loss: 0.4827, Val ROC-AUC: 0.8138, Val AP: 0.5981
Epoch [30/500], Train Loss: 0.4376, Val Loss: 0.4430, Val ROC-AUC: 0.8269, Val AP: 0.6188
Epoch [40/500], Train Loss: 0.4244, Val Loss: 0.4330, Val ROC-AUC: 0.8325, Val AP: 0.6271
Epoch [50/500], Train Loss: 0.4189, Val Loss: 0.4294, Val ROC-AUC: 0.8350, Val AP: 0.6281
Epoch [60/500], Train Loss: 0.4156, Val Loss: 0.4274, Val ROC-AUC: 0.8366, Val AP: 0.6310
Epoch [70/500], Train Loss: 0.4130, Val Loss: 0.4262, Val ROC-AUC: 0.8379, Val AP: 0.6335
Epoch [80/500], Train Loss: 0.4109, Val Loss: 0.4251, Val ROC-AUC: 0.8387, Val AP: 0.6363
Epoch [90/500], Train Loss: 0.4091, Val Loss: 0.4242, Val ROC-AUC: 0.8393, Val AP: 0.6372
Epoch [100/500], Train Loss: 0.4075, Val Loss: 0.4238, Val ROC-AUC: 0.8397, Val AP: 0.6391
Epoch [110/500], Train Loss: 0.4060, Val Loss: 0.4231, Val ROC-AUC: 0.8401, Val AP: 0.6403
Epoch [1

In [10]:
# Evaluate the model on the test set
model.eval()
with torch.no_grad():
    outputs = model(Xte_t)
    predicted = (outputs > 0.5).float()

# Convert PyTorch tensors to NumPy arrays for scikit-learn
y_true = yte_t.numpy()
y_pred = predicted.numpy()
y_prob = outputs.detach().numpy() # Get probabilities

# Calculate evaluation metrics
conf_matrix = confusion_matrix(y_true, y_pred)
precision = precision_score(y_true, y_pred)
recall = recall_score(y_true, y_pred)
f1 = f1_score(y_true, y_pred)
roc_auc = roc_auc_score(y_true, y_prob)
average_precision = average_precision_score(y_true, y_prob)


print("Confusion Matrix:")
print(conf_matrix)
print(f"\nPrecision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-score: {f1:.4f}")
print(f"ROC AUC: {roc_auc:.4f}")
print(f"Average Precision: {average_precision:.4f}")

Confusion Matrix:
[[975  58]
 [227 147]]

Precision: 0.7171
Recall: 0.3930
F1-score: 0.5078
ROC AUC: 0.8450
Average Precision: 0.6690


In [11]:
# Instantiate the AdaBoost Classifier
adaboost_model = AdaBoostClassifier(learning_rate=0.5, n_estimators=300, random_state=33)

# Train the model
adaboost_model.fit(Xtr, y_tr)

# Make predictions on the test set
y_pred_adaboost = adaboost_model.predict(Xte)
y_prob_adaboost = adaboost_model.predict_proba(Xte)[:, 1] # Get probabilities for the positive class

# Calculate evaluation metrics
conf_matrix_adaboost = confusion_matrix(y_test, y_pred_adaboost)
precision_adaboost = precision_score(y_test, y_pred_adaboost)
recall_adaboost = recall_score(y_test, y_pred_adaboost)
f1_adaboost = f1_score(y_test, y_pred_adaboost)
roc_auc_adaboost = roc_auc_score(y_test, y_prob_adaboost)
average_precision_adaboost = average_precision_score(y_test, y_prob_adaboost)

In [12]:
# Instantiate the RandomForest Classifier
rf_model = RandomForestClassifier(class_weight='balanced', max_depth=10,
                                  min_samples_leaf=2, random_state=33)

# Train the model
rf_model.fit(Xtr, y_tr)

# Make predictions on the test set
y_pred_rf = rf_model.predict(Xte)
y_prob_rf = rf_model.predict_proba(Xte)[:, 1] # Get probabilities for the positive class

# Calculate evaluation metrics
conf_matrix_rf = confusion_matrix(y_test, y_pred_rf)
precision_rf = precision_score(y_test, y_pred_rf)
recall_rf = recall_score(y_test, y_pred_rf)
f1_rf = f1_score(y_test, y_pred_rf)
roc_auc_rf = roc_auc_score(y_test, y_prob_rf)
average_precision_rf = average_precision_score(y_test, y_prob_rf)

In [13]:
comparison_data = {
    'Model': ['Neural Network', 'AdaBoost', 'Random Forest'],
    'Precision': [precision, precision_adaboost, precision_rf],
    'Recall': [recall, recall_adaboost, recall_rf],
    'F1-score': [f1, f1_adaboost, f1_rf],
    'ROC AUC': [roc_auc, roc_auc_adaboost, roc_auc_rf],
    'Average Precision': [average_precision, average_precision_adaboost, average_precision_rf]
}

comparison_df = pd.DataFrame(comparison_data)
comparison_df_sorted = comparison_df.sort_values(by='Average Precision', ascending=False)

print("Model Comparison Table (Sorted by Average Precision):")
display(comparison_df_sorted)

Model Comparison Table (Sorted by Average Precision):


,Model,Precision,Recall,F1-score,ROC AUC,Average Precision
0,Neural Network,0.717073,0.393048,0.507772,0.845032,0.669020
2,Random Forest,0.574786,0.719251,0.638955,0.844129,0.665707
1,AdaBoost,0.670103,0.521390,0.586466,0.844128,0.664520
